In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

from pympler import asizeof #Para liberar memoria
import gc

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, to_rgb

from sklearn import metrics
from sklearn.cluster import KMeans
import kneed
import random

from sklearn.preprocessing import StandardScaler

semilla = 8

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo )     

In [4]:
data_l.shape #(62130167, 16)

(62130167, 16)

In [5]:
inicio = time.time()
data_l = data_l.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )
data_l.reset_index(inplace=True)
data_l.rename(columns={'index': 'nuevo_indice'}, inplace=True)
data_l.drop(columns=['nuevo_indice'], inplace=True)
data_l['INDICE'] = data_l.index

#Casos de no análisis
data_no_grupo = data_l[ (data_l['GRUPO_SEL']==0) ].copy()
data_no_grupo['ATI_CJ_M1'] = np.nan
data_no_grupo['ATI_CJ_M2'] = np.nan
data_no_grupo['ATI_CJ_M3'] = np.nan

#Casos de análisis
data = data_l[ (data_l['GRUPO_SEL']==1) ].copy()

fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 14.243931770324707  segundos


In [6]:
data

,CEDULA_COD,ANIO,MES,SALARIO,SALARIO_SECTOR,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL,SBU,ID_SBU,INDICE
13,126,2006,1,161.67,161.67,PRI,1,1,5,161.670000,1,2010-12-01,2006-01-01,210.635,160.0,-1,13
14,126,2006,2,161.67,161.67,PRI,1,1,5,161.670000,1,2010-12-01,2006-01-01,210.635,160.0,-1,14
15,126,2006,3,161.67,161.67,PRI,1,1,5,161.670000,1,2010-12-01,2006-01-01,210.635,160.0,-1,15
16,126,2006,4,161.67,161.67,PRI,1,1,5,161.670000,1,2010-12-01,2006-01-01,210.635,160.0,-1,16
17,126,2006,5,161.67,161.67,PRI,1,1,5,161.670000,1,2010-12-01,2006-01-01,210.635,160.0,-1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62130162,23594465,2017,11,14045.72,14045.72,PRI,1,1,1,16205.699167,1,2018-03-01,2013-04-01,16604.299,375.0,-1,62130162
62130163,23594465,2017,12,14164.25,14164.25,PRI,1,1,1,16205.699167,1,2018-03-01,2013-04-01,16604.299,375.0,-1,62130163
62130164,23594465,2018,1,12903.21,12903.21,PRI,1,1,1,16205.699167,1,2018-03-01,2013-04-01,16604.299,386.0,-1,62130164
62130165,23594465,2018,2,42432.21,42432.21,PRI,1,1,1,16205.699167,1,2018-03-01,2013-04-01,16604.299,386.0,-1,62130165


In [ ]:
data_si = data[  (( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() 
data_no = data[ ~(( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() 

In [ ]:
#Solo se considerarn valores únicos, para evitar que al momento de generar el centroide exista conflicto de que
# varias observaciones tengan el mismo valor
data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': lambda x: list(set(x)) }).to_dict(orient='index')
data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': lambda x: list(set(x)) }).to_dict(orient='index')

In [ ]:
data_si_dic

In [ ]:
#Determinación del número de cluster según el método del codo y Silueta
def max_clust_silueta( silhouettes ):
    silhouettes = np.array(silhouettes)
    diferencias = np.abs(1 - silhouettes)
    indice_cercano = np.argmin( diferencias )
    return indice_cercano + 2

def num_cluster( data_si_dic, num, num_sample ):
    print('*' * 102)
    print('Determinación del número de clúster por el método del codo y silueta')
    inicio = time.time()
    
    sample_size = num_sample
    sample_keys = random.sample( list(data_si_dic.keys()), min(sample_size, len(data_si_dic)))
    
    data_filt = {key: data_si_dic[key] for key in sample_keys if key in data_si_dic}
    
    for ced  in data_filt:
        
        aux = np.array( data_filt[ced]['SALARIO']).reshape(-1, 1)
        
        if len(aux) <= 1:
            data_filt[ced]['num_codo'] = 1
            data_filt[ced]['max_silueta'] = 1
           
        
        tf = min(num, len(aux) - 1) + 1
        
        distorsion = []
        silueta = []

        for i in range( 2, tf ):
            km = KMeans( i, init = 'k-means++', n_init = 1, max_iter = 300, tol = 1e-4, random_state = semilla )
            clustering = km.fit_predict( aux )

            if( len( np.unique( clustering ) ) > 1):
                distorsion.append( km.inertia_ )
                silueta.append( metrics.silhouette_score(aux, clustering))
            
            else:
                distorsion.append(0)
                silueta.append(0)
    
        if( len(distorsion) > 1 and len(silueta) > 1 ):
            data_filt[ced]['num_codo'] = kneed.KneeLocator(range(2, tf), distorsion[:tf], curve="convex", 
                                                             direction="decreasing").elbow or 1
    
            data_filt[ced]['max_silueta'] =  max_clust_silueta( silueta )
        
        else:
            data_filt[ced]['num_codo'] = 1
            data_filt[ced]['max_silueta'] = 1
            
    

    fin = time.time()  
    print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    print('*' * 102)
    return data_filt

In [ ]:
data_filt = num_cluster( data_si_dic, 5, 400 ) #Analizar como sacar un buen muestreo de esto

In [ ]:
data_filt.keys()

In [ ]:
def graf_obs( data_filt, ced ):
    valores = data_filt[ced]['SALARIO']
    plt.figure( figsize=(10, 6))
    plt.scatter(range(len(valores)), valores, color='blue', marker='o')

    # Etiquetas y título
    plt.ylabel("USD")
    plt.title("Gráfico del salario único")

    plt.grid(True)
    plt.show()

def graf_obs_kmean( data_filt, ced):
    valores = np.array([[i + 1, valor] for i, valor in enumerate(data_filt[ced]['SALARIO'])])
    n = data_filt[ced]['num_codo']
    
    # Aplicar KMeans
    kmeans = KMeans(n_clusters=n, init='k-means++', n_init=10, random_state=42)
    y_pred = kmeans.fit_predict(valores)
    
    plt.figure(figsize=(10, 6))
    
    # Colormap para los clusters
    cmap = plt.get_cmap("jet")
    
    scatter = plt.scatter(valores[:, 0], valores[:, 1], c=y_pred, cmap=cmap, label='_nolegend_')
    
    
    plt.scatter(kmeans.cluster_centers_[:, 0],
                kmeans.cluster_centers_[:, 1],
                marker='^',
                c=list(range(0,n)),
                s=100,
                linewidth=2,
                edgecolor='black',
                cmap="jet",
                label=[f'Clúster {i+1}' for i in range(n)])
    plt.xlabel("Datos")
    plt.ylabel("USD")
    plt.title("Gráfico de Clústeres con Centroides")
    plt.grid(True)
    
     # Crear manejadores de leyenda para los puntos
    handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
    
    # Añadir la leyenda para los puntos y los centroides
    plt.legend( handles = handles, labels = [f'Clúster {i+1}' for i in range(n)], loc='best')

    plt.show()

In [ ]:
graf_obs( data_filt, 16346108 )

In [ ]:
graf_obs_kmean( data_filt, 16346108)

In [7]:
#Algortimos K-means
cedula_1 = data.groupby('CEDULA_COD')['NUM_SEC_MES'].apply( lambda x: (x != 1).any() )
cedula_dist = cedula_1[ cedula_1 ].index
ul = data[ ~data['CEDULA_COD'].isin( cedula_dist )]
ml = data[  data['CEDULA_COD'].isin( cedula_dist )]


In [8]:
print('Total de cédulas', data['CEDULA_COD'].nunique()) #442570
print('Cedulas que tienen un único sector', ul['CEDULA_COD'].nunique()) #373069
print('Cedulas que tienen más de un único sector', ml['CEDULA_COD'].nunique()) #69501

Total de cédulas 442570
Cedulas que tienen un único sector 373069
Cedulas que tienen más de un único sector 69501


In [41]:
#Normalización de datos-Z-core
print('*' * 40, 'Normalización Z-Score ', '*' * 40)
inicio = time.time()
scaler = StandardScaler()

# Aplicar la normalización usando transform
def data_norm( df1 ):
    df = df1.copy()
    me = df.groupby('CEDULA_COD')['SALARIO'].mean()
    st = df.groupby('CEDULA_COD')['SALARIO'].std()
    df['ME'] = df['CEDULA_COD'].map( me )
    df['ST'] = df['CEDULA_COD'].map( st )
    df['SALARIO_NORM'] = ( df['SALARIO'] - df['ME'] )  / df['ST'] 
    return df

ul =  data_norm( ul )
ml =  data_norm( ml )
fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
print('*' * 102)


**************************************** Normalización Z-Score  ****************************************
	Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 18.887230396270752  segundos
******************************************************************************************************


In [42]:
ul_dic = ul.groupby('CEDULA_COD').agg({'SALARIO': list,'SALARIO_NORM': list, 'INDICE': list}).to_dict(orient='index')
ml_dic = ml.groupby('CEDULA_COD').agg({'SALARIO': list,'SALARIO_NORM': list, 'INDICE': list}).to_dict(orient='index')

In [50]:
( 8.06-np.mean(ul_dic[126]['SALARIO']))/np.std(ul_dic[126]['SALARIO'])

-4.179024926830773

In [ ]:
n = 2
valores = np.array([[i + 1, valor] for i, valor in enumerate(data_si_dic[126]['SALARIO'])])
kmeans = KMeans( n_clusters = n, init = 'k-means++', n_init=10, random_state=42)
y_pred = kmeans.fit_predict(valores)

In [ ]:
ul_dic[3057]

In [ ]:
valores.shape[0]

In [ ]:
print('*' * 102)
print('Clasificación de las observaciones según cluster')
inicio = time.time()
n = 2
for ced in ul_dic:
    valores = np.array([[i + 1, valor] for i, valor in enumerate(ul_dic[ced]['SALARIO'])])
    
    if valores.shape[0] >= n:
        kmeans = KMeans( n_clusters = n, init = 'k-means++', n_init = 1, random_state = semilla)
        ul_dic[ced]['Ati'] = kmeans.fit_predict(valores)
    else:
        ul_dic[ced]['Ati'] = np.nan 
        
fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
print('*' * 102)

In [ ]:
from dask.distributed import Scheduler, Client
client = Client()
print(client.scheduler_info())

In [ ]:
scheduler_info = client.scheduler_info()
print("Scheduler Info:", scheduler_info)

# Ajusta según la estructura del diccionario
services = scheduler_info.get('services', {})
print("Services:", services)
for service, url in services.items():
    if 'tcp://' in url:
        print("Scheduler URL:", url)
        break

In [ ]:
import numpy as np
import time
from sklearn.cluster import KMeans
from joblib import Parallel, delayed

# Definir la función para aplicar KMeans a un conjunto de datos
def aplicar_kmeans(ced, valores, n, semilla):
    kmeans = KMeans( n_clusters = n, init='k-means++', n_init=1, random_state=semilla)
    etiquetas = kmeans.fit_predict(valores)
    return ced, etiquetas

print('*' * 102)
print('Clasificación de las observaciones según cluster')
inicio = time.time()

# Configuraciones
n = 2
semilla = 42  # Asegúrate de definir 'semilla' como en tu código original

# Crear listas de valores para paralelización
datos = [(ced, np.array([[i + 1, valor] for i, valor in enumerate(ul[ced]['SALARIO'])]), n, semilla) 
         for ced in ul]

# Aplicar KMeans en paralelo
resultados = Parallel(n_jobs=-1)(delayed(aplicar_kmeans)(*d) for d in datos)

# Almacenar resultados en el diccionario
for ced, etiquetas in resultados:
    ul[ced]['Ati'] = etiquetas
    print(ced)

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
print('*' * 102)